In [1]:
# !pip install tf-nightly

In [2]:
import tensorflow as tf
import tensorflow.keras.layers
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 1. SVM

In [3]:
df = pd.read_csv("/content/gdrive/My Drive/TSLA_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")

# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/TSLA_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/TSLA_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/TSLA_dataset02.csv")
print(df)

# plt.figure(figsize=(12,8))
# plt.plot(df['ALPHA'], label = 'ALPHA')
# plt.plot(df['BETA'], label = 'BETA')
# plt.legend()
# plt.show()


############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
215   50.0   25.0     poly         0.809524
127    1.0  100.0     poly         0.804762
187   25.0   50.0     poly         0.800000
155   10.0   50.0     poly         0.790476
247  100.0   25.0     poly         0.790476
..     ...    ...      ...              ...
186   25.0   50.0  sigmoid         0.366667
210   50.0   10.0  sigmoid         0.357143
178   25.0   10.0 

In [4]:
df = pd.read_csv("/content/gdrive/My Drive/SBUX_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")
# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/SBUX_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/SBUX_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/SBUX_dataset02.csv")
print(df)

############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
191   25.0  100.0     poly         0.933333
251  100.0   50.0     poly         0.928571
159   10.0  100.0     poly         0.928571
219   50.0   50.0     poly         0.923810
223   50.0  100.0     poly         0.923810
..     ...    ...      ...              ...
79     0.1    1.0     poly         0.528571
235  100.0    0.1     poly         0.528571
142   10.0    1.0 

In [5]:
df = pd.read_csv("/content/gdrive/My Drive/AMD_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")

# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/AMD_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/AMD_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/AMD_dataset02.csv")
print(df)

############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
251  100.0   50.0     poly         0.747619
187   25.0   50.0     poly         0.747619
219   50.0   50.0     poly         0.742857
223   50.0  100.0     poly         0.738095
155   10.0   50.0     poly         0.738095
..     ...    ...      ...              ...
154   10.0   50.0  sigmoid         0.304762
222   50.0  100.0  sigmoid         0.300000
218   50.0   50.0 

In [6]:
df = pd.read_csv("/content/gdrive/My Drive/INTC_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")

# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/INTC_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/INTC_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/INTC_dataset02.csv")
print(df)

############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
191   25.0  100.0     poly         0.814286
251  100.0   50.0     poly         0.814286
159   10.0  100.0     poly         0.809524
187   25.0   50.0     poly         0.809524
123    1.0   50.0     poly         0.804762
..     ...    ...      ...              ...
242  100.0   10.0  sigmoid         0.490476
142   10.0    1.0  sigmoid         0.390476
174   25.0    1.0 

In [7]:
df = pd.read_csv("/content/gdrive/My Drive/AMZN_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")

# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/AMZN_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/AMZN_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/AMZN_dataset02.csv")
print(df)

############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
221   50.0  100.0      rbf         0.714286
217   50.0   50.0      rbf         0.704762
253  100.0  100.0      rbf         0.704762
249  100.0   50.0      rbf         0.704762
245  100.0   25.0      rbf         0.700000
..     ...    ...      ...              ...
214   50.0   25.0  sigmoid         0.304762
246  100.0   25.0  sigmoid         0.304762
218   50.0   50.0 

In [8]:
df = pd.read_csv("/content/gdrive/My Drive/AAPL_dataset00.csv")
df.info()

from sklearn.model_selection import train_test_split

X = df.iloc[:,1:7].values
y = df['result02'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=0)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_clf = SVC(random_state=0)
parameters = {'kernel' : ['linear','rbf', 'sigmoid','poly'],
    'C' : [0.001,0.01,0.1,1,10,25,50,100],
    'gamma' : [0.001,0.01,0.1,1,10,25,50,100]}

grid_svm = GridSearchCV(svm_clf, param_grid = parameters, cv=5)
grid_svm.fit(X_train,y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
results = result.sort_values(by='mean_test_score', ascending=False)
print("\n",results)
print("\n최적 파라미터 값을 적용합니다 ...\n")

data = results.values[0]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc4 = SVC(C = C, gamma = gamma, kernel = kernel)
svc4.fit(X_train, y_train)

print("훈련 정확도 :", svc4.score(X_train, y_train))
print("예측 정확도 :", svc4.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction = svc4.predict(X)
confusion = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y, prediction, average="macro")
print("\nprecision_score :", precision)

recall = recall_score(y, prediction, average="macro")
print("recall_score :", recall)

f1_score0 = f1_score(y, prediction, average="macro")
print("f1_score :",f1_score0)

print("\n두 번째 파라미터 값을 적용합니다 ...\n")

data = results.values[4]
C = data[0]
gamma = data[1]
kernel = data[2]

from sklearn.svm import SVC

svc5 = SVC(C = C, gamma = gamma, kernel = kernel)
svc5.fit(X_train, y_train)

print("훈련 정확도 :", svc5.score(X_train, y_train))
print("예측 정확도 :", svc5.score(X_test, y_test))

from sklearn.metrics import confusion_matrix

prediction1 = svc5.predict(X)
confusion1 = confusion_matrix(y_true=y, y_pred=prediction)
print("\n",confusion1)

from sklearn.metrics import precision_score, recall_score, f1_score

precision1 = precision_score(y, prediction1, average="macro")
print("\nprecision_score :", precision)

recall1 = recall_score(y, prediction1, average="macro")
print("recall_score :", recall)

f1_score1 = f1_score(y, prediction1, average="macro")
print("f1_score :",f1_score1)
print("\n")

# 현재 값 예측시켜서 저장하기
df = pd.read_csv("/content/gdrive/My Drive/AAPL_dataset01.csv")

X = df.iloc[:,1:7].values

prediction = svc4.predict(X)
prediction1 = svc5.predict(X)

model4_pred = svc4.predict(X)
model5_pred = svc5.predict(X)
pred_04 = np.zeros(len(model4_pred))
pred_05 = np.zeros(len(model5_pred))

for i in range(len(model4_pred)) :
  pred_04[i] = model4_pred[i]
  pred_05[i] = model5_pred[i]

df['ALPHA'] = pd.Series(pred_04)
df['BETA'] = pd.Series(pred_05)
df = df[['date', 'ALPHA', 'BETA']]
df.to_csv("/content/gdrive/My Drive/AAPL_dataset02.csv", index=False)

df = pd.read_csv("/content/gdrive/My Drive/AAPL_dataset02.csv")
print(df)

############################################333# m4

import math

a1 = confusion[2,2]
a2 = confusion[1,2]
a3 = confusion[0,2]

pr = int(precision*100)
re = int(recall*100)
f1 = int(f1_score0 * 100)
ac = int(svc4.score(X_test, y_test)*100)

a4 = a1+a2+a3
h1 = a1/a4*100
h1 = math.trunc(h1)

r1 = a2/a4*100
r1 = math.trunc(r1)

list1 = [pr,re,ac,h1,r1]


if f1 >= 70 :

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h1 >= 70 :
        q1=("High Return")

    elif h1 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r1 >= 5 :
        q2=("Low Risk")

    elif r1 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 ALPHA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 ALPHA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')


################################ m5

b1 = confusion1[0,0]
b2 = confusion1[1,0]
b3 = confusion1[2,0]

pr1 = int(precision1*100)
re1 = int(recall1*100)
f1 = int(f1_score1*100)
ac1 = int(svc5.score(X_test, y_test)*100)

b4 = b1+b2+b3
h2 = b1/b4*100
h2 = math.trunc(h2)

r2 = b2/b4*100
r2 = math.trunc(r2)

list1 = [pr,re,ac,h2,r2]


if f1 >= 70 :

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 신뢰할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다. 
    
    ''' .format(f1,ac,pr,re,q2, q1))

elif f1 >= 60:

    if h2 >= 70 :
        q1=("High Return")

    elif h2 >= 60:
        q1=("Middle Return")

    else :
        q1=("Low Return")


    if r2 >= 5 :
        q2=("Low Risk")

    elif r2 >= 15 :
        q2=("Middle Risk")

    else :
        q2=("High Risk")

    print ('''
    
모델 BETA의 종합점수는 {} 이며 참고할 수 있는 수준 입니다.
정확도는 {} , 정밀도는 {} , 재현률은 {} 입니다.
특징은 {} {} 입니다.
    
    ''' .format(f1,ac,pr,re,q2, q1))


else : 
    
    print( '''
모델 BETA는 적합한 투자의견을 제시할 수 없습니다.
이 기업은 재무적, 정기적 요인이 아닌 비정기적, 
우발적 요인에 영향을 받는 주식일 확률이 높습니다.''')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      280 non-null    int64  
 1   pred1     280 non-null    float64
 2   pred1r    280 non-null    float64
 3   pred2     280 non-null    float64
 4   pred2r    280 non-null    float64
 5   pred3     280 non-null    float64
 6   pred3r    280 non-null    float64
 7   result01  280 non-null    float64
 8   result02  280 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 19.8 KB

          C  gamma   kernel  mean_test_score
245  100.0   25.0      rbf         0.780952
249  100.0   50.0      rbf         0.776190
179   25.0   10.0     poly         0.776190
217   50.0   50.0      rbf         0.771429
211   50.0   10.0     poly         0.771429
..     ...    ...      ...              ...
186   25.0   50.0  sigmoid         0.380952
222   50.0  100.0  sigmoid         0.376190
118    1.0   25.0 